# Loading Dataset

In [822]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

In [823]:
df =  pd.read_csv('datasets/nieruchomosci-online_dataset_raw.csv')

In [824]:
df.head(10)

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership
0,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żelechowskiego","Żelechowskiego, Bronowice, Kraków, małopolskie",899 000 zł,"51,70 m²","17 388,78 zł/m²",1,3.0,2,1980,tak,wtórny,własność
1,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Balicka","Balicka, Bronowice, Kraków, małopolskie",1 575 000 zł,125 m²,12 600 zł/m²,3,4.0,6,2004,w garażu podziemnym,wtórny,"własność, księga wieczysta"
2,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Zauchy","Zauchy, Górka Narodowa, Kraków, małopolskie",1 250 000 zł,"65,23 m²","19 162,96 zł/m²",2,5.0,3,2023,tak,wtórny,NaN
3,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Racławicka","Racławicka, Krowodrza, Kraków, małopolskie",740 250 zł,"49,35 m²",15 000 zł/m²,6,10.0,2,1950,parking publiczny / na ulicy,wtórny,NaN
4,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Pustynna","Pustynna, Kliny, Kraków, małopolskie",1 200 000 zł,"71,06 m²","16 887,14 zł/m²",parter,1.0,3,2013,garaż w bryle budynku,wtórny,"własność, księga wieczysta"
5,https://krakow.nieruchomosci-online.pl/mieszka...,Apartament Kraków,"Wola Justowska, Kraków, małopolskie",1 198 680 zł,"71,35 m²",16 800 zł/m²,parter,2.0,4,2023,garaż w bryle budynku,pierwotny (zobacz inne nowe mieszkania w Krako...,własność
6,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Przemiarki","Przemiarki, Ruczaj, Kraków, małopolskie",625 000 zł,40 m²,15 625 zł/m²,2,3.0,2,2000,garaż,wtórny,własność
7,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Felińskiego","Felińskiego, Górka Narodowa, Kraków, małopolskie",899 000 zł,"74,17 m²","12 120,80 zł/m²",2,5.0,3,2003,parking publiczny / na ulicy,wtórny,NaN
8,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. Lublańska","Lublańska 13, Prądnik Czerwony, Kraków, małopo...",1 050 000 zł,60 m²,17 500 zł/m²,3,4.0,3,2023,w garażu podziemnym,wtórny,"własność, księga wieczysta"
9,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. Szablowskiego","Szablowskiego, Bronowice, Kraków, małopolskie",599 000 zł,40 m²,14 975 zł/m²,parter,3.0,2,2001,tak,wtórny,"własność, księga wieczysta"


In [825]:
df.shape

(3949, 13)

# Data Cleaning

Steps:
* feature name change
* area and price from string to float + metrics
* missing values
* shifted data
* incorrect data cell
* feature type change

# Feature name change

In [826]:
df.rename(columns={'area': 'area (m^2)'}, inplace=True)
df.rename(columns={'price': 'price (zł)'}, inplace=True)
df.rename(columns={'no of floors/stores in the building': 'number of floors'}, inplace=True)
df.rename(columns={'floor/store': 'floor number'}, inplace=True)

In [827]:
df['price (zł)'] = df['price (zł)'].str.replace(',', '')

# Rozdzielenie wartości w kolumnie "price (zł)" na dwie nowe kolumny
price_split = df['price (zł)'].str.extract(r'([\d\s]+)\s+(.*)')
df['price_amount'] = price_split[0].str.replace(' ', '').astype(float)
df['currency'] = price_split[1]

# Zamiana przecinka na kropkę i konwersja na float w kolumnie "area (m^2)"
df['area_amount'] = df['area (m^2)'].str.replace(',', '.').str.extract(r'(\d+\.\d+|\d+)').astype(float)

# Wyciągnięcie jednostki metryki z kolumny "area (m^2)"
df['area_unit'] = df['area (m^2)'].str.extract(r'([a-zA-Z²]+)')

# Usunięcie wcześniejszych kolumn
df.drop(columns=['price (zł)', 'area (m^2)'], inplace=True)

In [828]:
unique_area_units = df['area_unit'].unique()
print("Unikalne jednostki metryczne:", unique_area_units)

unique_currencies = df['currency'].unique()
print("Unikalne waluty:", unique_currencies)

Unikalne jednostki metryczne: ['m²']
Unikalne waluty: ['zł' nan '€']


In [829]:
EURO_RATE = 4.3

# Warunkowe mnożenie kwoty przez stałą, jeśli waluta to euro (nie optymalne, ale za głupi na to jestem)
df['price_amount'] = df.apply(lambda row: row['price_amount'] * EURO_RATE if row['currency'] == 'euro' else row['price_amount'], axis=1)

# Zmiana waluty na "zł"
df.loc[df['currency'] == '€', 'currency'] = 'zł'

In [830]:
unique_area_units = df['area_unit'].unique()
print("Unikalne jednostki metryczne:", unique_area_units)

unique_currencies = df['currency'].unique()
print("Unikalne waluty:", unique_currencies)

Unikalne jednostki metryczne: ['m²']
Unikalne waluty: ['zł' nan]


In [831]:
df.head(20)

,url,name/title,address,price-per-area,floor number,number of floors,no of rooms,year of construction,parking space,market,form of ownership,price_amount,currency,area_amount,area_unit
0,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żelechowskiego","Żelechowskiego, Bronowice, Kraków, małopolskie","17 388,78 zł/m²",1,3.0,2,1980,tak,wtórny,własność,899000.0,zł,51.70,m²
1,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Balicka","Balicka, Bronowice, Kraków, małopolskie",12 600 zł/m²,3,4.0,6,2004,w garażu podziemnym,wtórny,"własność, księga wieczysta",1575000.0,zł,125.00,m²
2,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Zauchy","Zauchy, Górka Narodowa, Kraków, małopolskie","19 162,96 zł/m²",2,5.0,3,2023,tak,wtórny,NaN,1250000.0,zł,65.23,m²
3,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Racławicka","Racławicka, Krowodrza, Kraków, małopolskie",15 000 zł/m²,6,10.0,2,1950,parking publiczny / na ulicy,wtórny,NaN,740250.0,zł,49.35,m²
4,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Pustynna","Pustynna, Kliny, Kraków, małopolskie","16 887,14 zł/m²",parter,1.0,3,2013,garaż w bryle budynku,wtórny,"własność, księga wieczysta",1200000.0,zł,71.06,m²
5,https://krakow.nieruchomosci-online.pl/mieszka...,Apartament Kraków,"Wola Justowska, Kraków, małopolskie",16 800 zł/m²,parter,2.0,4,2023,garaż w bryle budynku,pierwotny (zobacz inne nowe mieszkania w Krako...,własność,1198680.0,zł,71.35,m²
6,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Przemiarki","Przemiarki, Ruczaj, Kraków, małopolskie",15 625 zł/m²,2,3.0,2,2000,garaż,wtórny,własność,625000.0,zł,40.00,m²
7,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Felińskiego","Felińskiego, Górka Narodowa, Kraków, małopolskie","12 120,80 zł/m²",2,5.0,3,2003,parking publiczny / na ulicy,wtórny,NaN,899000.0,zł,74.17,m²
8,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. Lublańska","Lublańska 13, Prądnik Czerwony, Kraków, małopo...",17 500 zł/m²,3,4.0,3,2023,w garażu podziemnym,wtórny,"własność, księga wieczysta",1050000.0,zł,60.00,m²
9,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. Szablowskiego","Szablowskiego, Bronowice, Kraków, małopolskie",14 975 zł/m²,parter,3.0,2,2001,tak,wtórny,"własność, księga wieczysta",599000.0,zł,40.00,m²


In [832]:
unique_area_units = df['form of ownership'].unique()
print("Unikalne jednostki metryczne:", unique_area_units)

unique_currencies = df['market'].unique()
print("Unikalne waluty:", unique_currencies)

Unikalne jednostki metryczne: ['własność' 'własność, księga wieczysta' nan
 'spółdzielcze własnościowe, księga wieczysta'
 'pierwotny (oferta dewelopera)' 'udział w KW' 'spółdzielcze własnościowe'
 'udział ze wskazaniem, KW na budynku' 'księga wieczysta' 'inna' 'udział'
 'inna, księga wieczysta' 'Udział ze wskazaniem' 'Udział'
 'udział, księga wieczysta' 'udział ze wskazaniem' 'Chce dopłacić' 'tak']
Unikalne waluty: ['wtórny' 'pierwotny (zobacz inne nowe mieszkania w Krakowie)' 'dostępne'
 'II kwartał 2024' 'III kwartał 2024' 'IV kwartał 2022, oddana do użytku'
 'II kwartał 2025' 'III kwartał 2025' 'październik 2023, zrealizowana'
 'czerwiec 2023, zrealizowana' 'II kwartał 2023, oddana do użytku'
 'sierpień 2024' 'październik 2025' 'IV kwartał 2024' nan 'zarezerwowane'
 'sprzedane' 'I kwartał 2024' 'I kwartał 2025' 'IV kwartał 2025'
 'lipiec 2025' '2025' 'III kwartał 2023, oddana do użytku'
 'kwiecień 2023, oddana do użytku' 'marzec 2024' '1 miesiąc' 'natychmiast']


## Missing Values

| Attribute               | Value |
|-------------------------|-------|
| url                     | 0     |
| name/title              | 0     |
| address                 | 101   |
| price-per-area          | 29    |
| floor number            | 0     |
| number of floors        | 239   |
| no of rooms             | 0     |
| year of construction    | 0     |
| parking space           | 0     |
| market                  | 31    |
| form of ownership       | 2461  |
| price_amount            | 29    |
| currency                | 29    |
| area_amount             | 0     |
| area_unit               | 0     |

In [833]:
# Nowe obliczenie price-per-area bo tamtemu nie ufam
df['price_per_area (m2/zł)'] = round(df['price_amount'] / df['area_amount'], 2)
df.drop(columns=['price-per-area'], inplace=True)

In [834]:
# Wywalam wszystkie kolumny, w których nie mam danych odnośnie ceny
missing_currency = df[df['currency'].isnull()]
missing_currency

,url,name/title,address,floor number,number of floors,no of rooms,year of construction,parking space,market,form of ownership,price_amount,currency,area_amount,area_unit,price_per_area (m2/zł)
26,https://wieliczka.nieruchomosci-online.pl/nowe...,Nowe mieszkanie ul. Magnoliowa,NaN,wrzesień 2024,NaN,1,3,-,dostępne,NaN,NaN,NaN,57.83,m²,NaN
27,https://wieliczka.nieruchomosci-online.pl/nowe...,Nowe mieszkanie ul. Magnoliowa,NaN,wrzesień 2024,NaN,1,3,naziemne,dostępne,NaN,NaN,NaN,57.83,m²,NaN
28,https://wieliczka.nieruchomosci-online.pl/nowe...,Nowe mieszkanie ul. Magnoliowa,NaN,wrzesień 2024,NaN,1,3,naziemne,dostępne,NaN,NaN,NaN,57.83,m²,NaN
45,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Gotyk, ul. Grabczaka 8",NaN,III kwartał 2024,NaN,suterena,3,garaż,dostępne,NaN,NaN,NaN,58.36,m²,NaN
172,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Kleparz, ul. Długa 24",NaN,luty 2024,NaN,4,2,garaż,dostępne,NaN,NaN,NaN,39.84,m²,NaN
175,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Kleparz, ul. Długa 24",NaN,luty 2024,NaN,4,2,garaż,dostępne,NaN,NaN,NaN,57.45,m²,NaN
177,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Kleparz, ul. Długa 24",NaN,luty 2024,NaN,4,2,garaż,dostępne,NaN,NaN,NaN,38.93,m²,NaN
220,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Olszyny, ul. Petrażyckiego",NaN,wrzesień 2024,NaN,1,4,naziemne,dostępne,NaN,NaN,NaN,93.00,m²,NaN
221,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Olszyny, ul. Petrażyckiego",NaN,wrzesień 2024,NaN,parter,3,naziemne,zarezerwowane,NaN,NaN,NaN,55.00,m²,NaN
226,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Olszyny, ul. Petrażyckiego",NaN,wrzesień 2024,NaN,1,4,naziemne,dostępne,NaN,NaN,NaN,93.00,m²,NaN


In [835]:
# Brak przesunięcia w danych z missing price więc usuwam wszystkie, które nie posiadają wyceny
missing_currency['address']

26                                             NaN
27                                             NaN
28                                             NaN
45                                             NaN
172                                            NaN
175                                            NaN
177                                            NaN
220                                            NaN
221                                            NaN
226                                            NaN
257                                            NaN
273                                            NaN
274                                            NaN
276                                            NaN
304                                            NaN
305                                            NaN
309                                            NaN
318                                            NaN
321                                            NaN
323                            

In [836]:
df.dropna(subset=['price_amount'], inplace=True)

In [837]:
# Wywalam całą kolumnę form of ownership bo ma ponad 2500 pustych wierszy
df.drop(columns=['form of ownership'], inplace=True)

In [838]:
# Z racji tego, że brakuje tylko 29 marketów na prawie 4000 mieszkań to braki uzupełnię średnia.
# TODO, może się tak zdażyć, że floor number będzie większe niż number of floors, więc trzeba też napisać kod, który zmienia tą wartość jeśli floor number jest większe

number_of_floors_na = df['number of floors'].unique()
print("Unikalne wartości w number of floors:", number_of_floors_na)

Unikalne wartości w number of floors: [ 3.  4.  5. 10.  1.  2.  7.  6.  9. nan  8. 14. 11. 15. 16. 13. 12. 45.]


In [839]:
mean_number_of_floors = round(df['number of floors'].mean())
fill_value = {'number of floors': mean_number_of_floors}
df.fillna(value=fill_value, inplace=True)

print("Średnia wartość w mean number of floors:", mean_number_of_floors)

Średnia wartość w mean number of floors: 5


In [840]:
# TODO, TU JEST PROBLEM (którego rozwiązania teraz nie wymyśle WIĘC żeby ruszyć dalej pozbywam się tej kolumny. (prawdopodobnie się jej pozbędziemy)

market = df['market'].unique()
print("Unikalne jednostki metryczne:", market)

df.drop(columns=['market'], inplace=True)

Unikalne jednostki metryczne: ['wtórny' 'pierwotny (zobacz inne nowe mieszkania w Krakowie)'
 'II kwartał 2024' 'III kwartał 2024' 'IV kwartał 2022, oddana do użytku'
 'II kwartał 2025' 'III kwartał 2025' 'październik 2023, zrealizowana'
 'czerwiec 2023, zrealizowana' 'II kwartał 2023, oddana do użytku'
 'sierpień 2024' 'październik 2025' 'IV kwartał 2024' nan 'I kwartał 2024'
 'I kwartał 2025' 'IV kwartał 2025' 'lipiec 2025' '2025'
 'III kwartał 2023, oddana do użytku' 'kwiecień 2023, oddana do użytku'
 'marzec 2024' '1 miesiąc' 'natychmiast']


In [841]:
df.isnull().sum()

url                        0
name/title                 0
address                   74
floor number               0
number of floors           0
no of rooms                0
year of construction       0
parking space              0
price_amount               0
currency                   0
area_amount                0
area_unit                  0
price_per_area (m2/zł)     0
dtype: int64

## Shifted data / String data

In [842]:
unique_parking = df['parking space'].unique()
print("Unikalne wartości 'parking':", unique_parking)

Unikalne wartości 'parking': ['tak' 'w garażu podziemnym' 'parking publiczny / na ulicy'
 'garaż w bryle budynku' 'garaż' '-' 'przynależne na ulicy'
 'przynależne na terenie ogrodzonym' 'możliwość wykupienia' 'naziemne'
 'garaż wolnostojący' 'parking strzeżony w pobliżu' 'wiata garażowa']


In [843]:
# TODO, dokonuję uproszczeń, nie wiem czy tak akceptujecie

parking_counts = df['parking space'].value_counts()
parking_counts

parking space
-                                    965
w garażu podziemnym                  590
tak                                  572
parking publiczny / na ulicy         564
garaż                                439
przynależne na ulicy                 264
garaż wolnostojący                   161
garaż w bryle budynku                116
przynależne na terenie ogrodzonym     97
parking strzeżony w pobliżu           84
możliwość wykupienia                  39
naziemne                              17
wiata garażowa                        12
Name: count, dtype: int64

In [844]:
def map_to_boolean(value):
    if value == '-':
        return False
    else:
        return True
    
df['parking space'] = df['parking space'].map(map_to_boolean)

In [845]:
unique_construction_year = df['year of construction'].unique()
print("Unikalne wartości 'year of construction':", unique_construction_year)

Unikalne wartości 'year of construction': ['1980' '2004' '2023' '1950' '2013' '2000' '2003' '2001' '2017' '1962'
 '1970' '1985' '1983' '-' '2012' '2007' '1981' '2021' '1974' '1991' '2019'
 '1967' '1800' '4' '2010' '3' '5' '1972' '2024' '2009' '2011' '2022'
 '1989' '1920' '2014' '1957' '1958' '2018' '2' '1913' '2020' '1952' '2015'
 '1908' '1965' '1960' '2008' '2025' '1955' '1997' '1993' '1926' '1880'
 '1905' '1906' '1925' '1990' '1954' '1509' '1930' '1998' '1933' '2006'
 '1900' '1975' '2016' '1976' '1907' '1894' '2002' '1977' '1996' '1934'
 '1890' '1935' '1999' '1953' '1961' '1936' '1978' '1982' '1932' '1896'
 '1910' '1979' '2026' '6' '2005' '1914' '1968' '1992' '1898' '1750' '1939'
 '1973' '1937' '1969' '1995' '1988' '1964' '1938' '1971' '1987' '1928'
 '1912' '1956' '1915' '1986' '1940' '1942' '1923' '1886' '1966' '1892'
 '1959' '1893' '1911' '1929' '1899' '1891' '1850' '1500' '1909' '1350'
 '1922' '1320' '1870' '1790' '1024' '1994' '1946' '1921' '1887' '1878'
 '1895' '1694' '1832' '18

In [846]:
df['year of construction'] = pd.to_numeric(df['year of construction'], errors='coerce')

missing_below_1200 = df[df['year of construction'] < 1200]

count_missing_below_1200 = missing_below_1200.shape[0]

print("Number of values below 1200 and NaN values in 'year of construction':", count_missing_below_1200)

Number of values below 1200 and NaN values in 'year of construction': 75


In [847]:
# Obliczenie średniej z wartości dla year of construction
mean_year_all = int(round(df['year of construction'].mean()))

print(f"Average year of construction: {mean_year_all}")

# Zastąpienie NaN i wartości poniżej 1200 wskazanymi polami wartością średnią
df.loc[df['year of construction'].isna() | (df['year of construction'] < 1200), 'year of construction'] = mean_year_all

Average year of construction: 1960


In [848]:
df.head(50)

,url,name/title,address,floor number,number of floors,no of rooms,year of construction,parking space,price_amount,currency,area_amount,area_unit,price_per_area (m2/zł)
0,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żelechowskiego","Żelechowskiego, Bronowice, Kraków, małopolskie",1,3.0,2,1980.0,True,899000.0,zł,51.70,m²,17388.78
1,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Balicka","Balicka, Bronowice, Kraków, małopolskie",3,4.0,6,2004.0,True,1575000.0,zł,125.00,m²,12600.00
2,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Zauchy","Zauchy, Górka Narodowa, Kraków, małopolskie",2,5.0,3,2023.0,True,1250000.0,zł,65.23,m²,19162.96
3,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Racławicka","Racławicka, Krowodrza, Kraków, małopolskie",6,10.0,2,1950.0,True,740250.0,zł,49.35,m²,15000.00
4,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Pustynna","Pustynna, Kliny, Kraków, małopolskie",parter,1.0,3,2013.0,True,1200000.0,zł,71.06,m²,16887.14
5,https://krakow.nieruchomosci-online.pl/mieszka...,Apartament Kraków,"Wola Justowska, Kraków, małopolskie",parter,2.0,4,2023.0,True,1198680.0,zł,71.35,m²,16800.00
6,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Przemiarki","Przemiarki, Ruczaj, Kraków, małopolskie",2,3.0,2,2000.0,True,625000.0,zł,40.00,m²,15625.00
7,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Felińskiego","Felińskiego, Górka Narodowa, Kraków, małopolskie",2,5.0,3,2003.0,True,899000.0,zł,74.17,m²,12120.80
8,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. Lublańska","Lublańska 13, Prądnik Czerwony, Kraków, małopo...",3,4.0,3,2023.0,True,1050000.0,zł,60.00,m²,17500.00
9,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. Szablowskiego","Szablowskiego, Bronowice, Kraków, małopolskie",parter,3.0,2,2001.0,True,599000.0,zł,40.00,m²,14975.00


In [849]:
unique_no_rooms = df['no of rooms'].unique()
print("Unikalne wartości 'no of rooms':", unique_no_rooms)

Unikalne wartości 'no of rooms': ['2' '6' '3' '4' '5' 'parter' '1' '7' '9' '8' '10' '-' '28' '15']


In [850]:
room_counts = df['no of rooms'].value_counts()
room_counts

no of rooms
2         1391
3         1316
4          592
1          397
5          142
6           28
parter      20
7           11
-           10
9            5
8            4
10           2
28           1
15           1
Name: count, dtype: int64

In [851]:
# Zamiana '-' na modalną wartość
mode_value = df['no of rooms'].mode()[0]  # Pobranie modalnej wartości
df['no of rooms'] = df['no of rooms'].replace('-', mode_value)
df['no of rooms'] = df['no of rooms'].replace('parter', mode_value)

# Konwersja kolumny 'no of rooms' na typ int
df['no of rooms'] = pd.to_numeric(df['no of rooms'], errors='coerce')

In [852]:
unique_no_floor = df['floor number'].unique()
print("Unikalne wartości 'number of rooms':", unique_no_floor)

Unikalne wartości 'number of rooms': ['1' '3' '2' '6' 'parter' '7' '4' '9' 'II kwartał 2024' 'III kwartał 2024'
 '13' '5' 'oddana do użytku' 'II kwartał 2025' 'III kwartał 2025'
 'zrealizowana' '8' 'sierpień 2024' 'październik 2025' '15' 'suterena'
 'IV kwartał 2024' '11' 'I kwartał 2024' '14' '10' '-' 'I kwartał 2025'
 'IV kwartał 2025' 'lipiec 2025' '2025' '12' 'marzec 2024' '21']


In [853]:
no_floor_count = df['floor number'].value_counts()
no_floor_count

floor number
1                   896
parter              730
2                   662
3                   550
4                   435
5                   213
7                   106
6                   104
8                    39
-                    37
9                    21
10                   18
oddana do użytku      9
IV kwartał 2024       9
II kwartał 2025       8
III kwartał 2025      8
14                    8
I kwartał 2024        8
suterena              8
12                    7
zrealizowana          6
11                    5
13                    4
lipiec 2025           3
marzec 2024           3
IV kwartał 2025       3
2025                  3
sierpień 2024         3
I kwartał 2025        3
październik 2025      3
II kwartał 2024       3
15                    2
III kwartał 2024      2
21                    1
Name: count, dtype: int64

In [854]:
# Zamiana 'parter' na 0 i 'suterena' na -1
df['floor number'] = df['floor number'].replace({'parter': 0, 'suterena': -1})

# Konwersja kolumny na numeryczną, zamieniając nieprawidłowe wartości na NaN
df['floor number'] = pd.to_numeric(df['floor number'], errors='coerce')

# Znalezienie modalnej wartości
mode_value = df['floor number'].mode()[0]

# Zastąpienie NaN modalną wartością
df['floor number'] = df['floor number'].fillna(mode_value)

In [855]:
df.head(5)

,url,name/title,address,floor number,number of floors,no of rooms,year of construction,parking space,price_amount,currency,area_amount,area_unit,price_per_area (m2/zł)
0,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żelechowskiego","Żelechowskiego, Bronowice, Kraków, małopolskie",1.0,3.0,2,1980.0,True,899000.0,zł,51.70,m²,17388.78
1,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Balicka","Balicka, Bronowice, Kraków, małopolskie",3.0,4.0,6,2004.0,True,1575000.0,zł,125.00,m²,12600.00
2,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Zauchy","Zauchy, Górka Narodowa, Kraków, małopolskie",2.0,5.0,3,2023.0,True,1250000.0,zł,65.23,m²,19162.96
3,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Racławicka","Racławicka, Krowodrza, Kraków, małopolskie",6.0,10.0,2,1950.0,True,740250.0,zł,49.35,m²,15000.00
4,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Pustynna","Pustynna, Kliny, Kraków, małopolskie",0.0,1.0,3,2013.0,True,1200000.0,zł,71.06,m²,16887.14


## Address from Title

In [856]:
import re

# Wyrażenie regularne do wyciągnięcia tekstu po 'ul.'
pattern = r'ul\.\s*(.+)'

# Wykorzystanie metody str.extract() do wyciągnięcia tekstu i dodanie do nowej kolumny
df['address title'] = df['name/title'].str.extract(pattern, flags=re.IGNORECASE)

In [857]:
df.head(50)

,url,name/title,address,floor number,number of floors,no of rooms,year of construction,parking space,price_amount,currency,area_amount,area_unit,price_per_area (m2/zł),address title
0,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żelechowskiego","Żelechowskiego, Bronowice, Kraków, małopolskie",1.0,3.0,2,1980.0,True,899000.0,zł,51.70,m²,17388.78,Żelechowskiego
1,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Balicka","Balicka, Bronowice, Kraków, małopolskie",3.0,4.0,6,2004.0,True,1575000.0,zł,125.00,m²,12600.00,Balicka
2,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Zauchy","Zauchy, Górka Narodowa, Kraków, małopolskie",2.0,5.0,3,2023.0,True,1250000.0,zł,65.23,m²,19162.96,Zauchy
3,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Racławicka","Racławicka, Krowodrza, Kraków, małopolskie",6.0,10.0,2,1950.0,True,740250.0,zł,49.35,m²,15000.00,Racławicka
4,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Pustynna","Pustynna, Kliny, Kraków, małopolskie",0.0,1.0,3,2013.0,True,1200000.0,zł,71.06,m²,16887.14,Pustynna
5,https://krakow.nieruchomosci-online.pl/mieszka...,Apartament Kraków,"Wola Justowska, Kraków, małopolskie",0.0,2.0,4,2023.0,True,1198680.0,zł,71.35,m²,16800.00,NaN
6,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Przemiarki","Przemiarki, Ruczaj, Kraków, małopolskie",2.0,3.0,2,2000.0,True,625000.0,zł,40.00,m²,15625.00,Przemiarki
7,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Felińskiego","Felińskiego, Górka Narodowa, Kraków, małopolskie",2.0,5.0,3,2003.0,True,899000.0,zł,74.17,m²,12120.80,Felińskiego
8,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. Lublańska","Lublańska 13, Prądnik Czerwony, Kraków, małopo...",3.0,4.0,3,2023.0,True,1050000.0,zł,60.00,m²,17500.00,Lublańska
9,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. Szablowskiego","Szablowskiego, Bronowice, Kraków, małopolskie",0.0,3.0,2,2001.0,True,599000.0,zł,40.00,m²,14975.00,Szablowskiego


In [858]:
# Wyciągnięcie unikalnych wartości z kolumny 'address title'
unique_address_titles = df['address title'].unique()

# Policzenie liczby unikalnych wartości
count_unique_address_titles = len(unique_address_titles)

# Wyświetlenie unikalnych wartości i ich liczby
print("Unikalne wartości z kolumny 'address title':", unique_address_titles)
print("Liczba unikalnych wartości:", count_unique_address_titles)

Unikalne wartości z kolumny 'address title': ['Żelechowskiego' 'Balicka' 'Zauchy' 'Racławicka' 'Pustynna' nan
 'Przemiarki' 'Felińskiego' 'Lublańska' 'Szablowskiego' 'Cystersów'
 'Ułanów' 'Bartla' 'Bojki' 'Zawodzie' 'Obozowa' 'Beskidzka'
 'Cieplińskiego' 'Zwierzyniecka' 'Siewna' 'Przybyszewskiego' 'Dworcowa'
 'Lentza' 'Sławkowska' 'Okólna' 'Domagały' 'Stańczyka' 'Grabczaka 8'
 'Wygoda' 'Zakrzowiecka' 'Hamernia' 'Kordiana' 'Lubocka' 'Rydlówka'
 'Pszczelna' 'Gajówka' 'Władysława Łokietka' 'Sosnowiecka' 'Wawelska'
 'Bochnaka' 'Piaskowa 43' 'Okulickiego' 'Małysiaka' 'Pachońskiego'
 'Republiki Korczakowskiej 21' 'Słomczyńskiego' 'Lubicz' 'Laurowa'
 'Meiera' 'Kocmyrzowska 3A' 'Zbożowa' 'Truszkowskiego' 'Lea' 'Bunscha'
 'Bieńczycka' 'Sławka' 'Polonijna' 'Rydygiera' 'Rejtana' 'Brązownicza'
 'Szlak' 'Górników' 'Nuszkiewicza' 'Na Błonie' 'Klemensiewicza'
 'Chełmońskiego' 'Buszka' 'Czeczów' 'Królowej Jadwigi' 'Głowackiego'
 'Masarska' 'Raciborska' 'Turniejowa 59' 'Wizjonerów' 'Lipska'
 'Czarnogór

# Feature Engineering